In [1]:
# Download the corresponding PyTorch Geometric module
%%capture
"""
Assign to TORCH with what you get from the cell above, E.g., export TORCH=1.13.1+cu113
"""
%env TORCH=2.1.0+cu118
!pip install torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install torch-geometric

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch_geometric
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv
from torch_geometric.data import DataLoader
import torch.nn.functional as F

/usr/local/lib/python3.11/dist-packages/torch_geometric/typing.py:86: UserWarning: An issue occurred while importing 'torch-scatter'. Disabling its usage. Stacktrace: /usr/local/lib/python3.11/dist-packages/torch_scatter/_version_cuda.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev
  warnings.warn(f"An issue occurred while importing 'torch-scatter'. "
/usr/local/lib/python3.11/dist-packages/torch_geometric/typing.py:124: UserWarning: An issue occurred while importing 'torch-sparse'. Disabling its usage. Stacktrace: /usr/local/lib/python3.11/dist-packages/torch_sparse/_version_cuda.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev
  warnings.warn(f"An issue occurred while importing 'torch-sparse'. "


In [3]:
# # Load the Cora dataset
dataset = Planetoid(root='/tmp/Cora', name='Cora')
data = dataset[0]


# Check if CUDA is available and use it
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Move the data to the device (GPU if available)
data = data.to(device)

Processing...
Done!


In [4]:
class Simple_GCN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(Simple_GCN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        self.lin = torch.nn.Linear(hidden_dim, output_dim)


    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = self.conv2(x, edge_index)
        x = self.lin(x)
        return x

    def get_ebd(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = self.conv2(x, edge_index)
        return x


In [5]:
# Training the model
def train(model, data, optimizer, criterion, epochs=200):
    model.train()
    for epoch in range(epochs):
        optimizer.zero_grad()
        # Get the node features (data.x) and edge indices (data.edge_index)
        out = model(data.x, data.edge_index)
        loss = criterion(out[data.train_mask], data.y[data.train_mask])
        loss.backward()
        optimizer.step()

        if epoch % 10 == 0:
            print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}')

# Evaluate the model
def evaluate(model, data):
    model.eval()
    with torch.no_grad():
        out = model(data.x, data.edge_index)
        pred = out.argmax(dim=1)
        correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
        acc = correct / data.test_mask.sum()
        return acc.item()

from sklearn.metrics import f1_score
import numpy as np
# Replace the original `test()` function
def evaluate_dice(model, data):
    model.eval()
    with torch.no_grad():
        out = model(data.x, data.edge_index)
        pred = out.argmax(dim=1)

        y_true = data.y[data.test_mask].cpu().numpy()
        y_pred = pred[data.test_mask].cpu().numpy()

        # Dice score is equivalent to macro F1 score for multiclass
        dice = f1_score(y_true, y_pred, average='macro')
        return dice

In [6]:
# Model hyperparameters
in_channels = dataset.num_node_features  # 1433 (Cora input feature size)
hidden_channels = 64
out_channels = dataset.num_classes  # 7 (Cora has 7 classes)

# Initialize the model
model = Simple_GCN(in_channels, hidden_channels, out_channels).to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)


In [32]:
# Train the model
train(model, data, optimizer, criterion, epochs=50)

# Evaluate the model
acc = evaluate(model, data)
print(f'Test accuracy: {acc:.4f}')

dice = evaluate_dice(model, data)
print(f'Test dice: {dice:.4f}')

Epoch 1/50, Loss: 0.0031
Epoch 11/50, Loss: 0.0031
Epoch 21/50, Loss: 0.0030
Epoch 31/50, Loss: 0.0030
Epoch 41/50, Loss: 0.0030
Test accuracy: 0.7980
Test dice: 0.7907


In [12]:
from torch_geometric.utils import to_dense_adj
from Perturbe_Algs import Attacks
from copy import deepcopy

# Node features (X)
x = data.x  # Shape: [num_nodes, num_features]
# Adjacency matrix (A)
A = to_dense_adj(data.edge_index)[0]  # Shape: [num_nodes, num_nodes]
K = (data.x) @ (data.x).T
m = 1000

In [14]:
attack_instance = Attacks(A,K,m, alpha = 50, max_iter=200, filter = 'adj_norm_self_loop').to(device)

pgdavg_dataset = deepcopy(data)
pgdwst_dataset = deepcopy(data)

A_pgd_avg = attack_instance.Prob_PGD().clone().detach()
A_pgd_wst = attack_instance.Wst_PGD().clone().detach()

pgdavg_dataset.edge_index = torch.tensor(A_pgd_avg).nonzero(as_tuple=False).t().contiguous()
pgdwst_dataset.edge_index = torch.tensor(A_pgd_wst).nonzero(as_tuple=False).t().contiguous()


<ipython-input-14-005b8a245116>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  pgdavg_dataset.edge_index = torch.tensor(A_pgd_avg).nonzero(as_tuple=False).t().contiguous()
<ipython-input-14-005b8a245116>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  pgdwst_dataset.edge_index = torch.tensor(A_pgd_wst).nonzero(as_tuple=False).t().contiguous()


In [33]:

num_rep = 5
list_ebd_random = []
list_ebd_pgdavg = []
list_ebd_pgdwst = []


list_acc_random = []
list_acc_pgdavg = []
list_acc_pgdwst = []

for i in range(num_rep):


  A_random = attack_instance.randomAttack().clone().detach()

  random_dataset = deepcopy(data)
  random_dataset.edge_index = torch.tensor(A_random).nonzero(as_tuple=False).t().contiguous()

  acc = evaluate(model, data)
  dice = evaluate_dice(model, data)
  acc_random = evaluate(model, random_dataset)
  dice_random = evaluate_dice(model, random_dataset)

  acc_pgdavg = evaluate(model, pgdavg_dataset)
  dice_pgdavg = evaluate_dice(model, pgdavg_dataset)

  acc_pgdwst = evaluate(model, pgdwst_dataset)
  dice_pgdwst = evaluate_dice(model, pgdwst_dataset)


  list_acc_random.append(acc_random)
  list_acc_pgdavg.append(acc_pgdavg)
  list_acc_pgdwst.append(acc_pgdwst)


  ebd = model.get_ebd(data.x, data.edge_index)
  ebd_random = model.get_ebd(random_dataset.x, random_dataset.edge_index)
  ebd_pgdavg = model.get_ebd(pgdavg_dataset.x, pgdavg_dataset.edge_index)
  ebd_pgdwst = model.get_ebd(pgdwst_dataset.x, pgdwst_dataset.edge_index)


  p_ebd_random = torch.norm(ebd - ebd_random, p='fro')
  p_ebd_pgdavg = torch.norm(ebd - ebd_pgdavg, p='fro')
  p_ebd_pgdwst = torch.norm(ebd - ebd_pgdwst, p='fro')

  list_ebd_random.append(p_ebd_random.item())
  list_ebd_pgdavg.append(p_ebd_pgdavg.item())
  list_ebd_pgdwst.append(p_ebd_pgdwst.item())


<ipython-input-33-f05164b2ed31>:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  random_dataset.edge_index = torch.tensor(A_random).nonzero(as_tuple=False).t().contiguous()
<ipython-input-33-f05164b2ed31>:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  random_dataset.edge_index = torch.tensor(A_random).nonzero(as_tuple=False).t().contiguous()
<ipython-input-33-f05164b2ed31>:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  random_dataset.edge_index = torch.tensor(A_random).nonzero(as_tuple=False).t().contiguous()
<ipython-input-33-f051

In [34]:
print(np.mean(list_acc_random), np.std(list_acc_random))
print(np.mean(list_acc_pgdwst), np.std(list_acc_pgdwst))
print(np.mean(list_acc_pgdavg), np.std(list_acc_pgdavg))


0.7740000009536743 0.005727131705758155
0.7789999842643738 0.0
0.7720000147819519 0.0


In [35]:
print(np.mean(list_ebd_random), np.std(list_ebd_random))
print(np.mean(list_ebd_pgdwst), np.std(list_ebd_pgdwst))
print(np.mean(list_ebd_pgdavg), np.std(list_ebd_pgdavg))


54.781385803222655 0.41286593071133626
66.34374694824218 3.7376247295885895e-06
63.2568000793457 1.868812364794295e-06
